In [1]:
from preprocessing import *
from representation import ast, dimacs
from sat_solvers import CDCL, CDCLOptions, HeuristicType

# Solving the problem of Γ ⊨ A using CDCL

#### The problem:
* Γ is a Theory (a Finite Set of Propositional Formulas);
* A is a Propositional Formula;
* We are asked to determine whether A is a Logical Consequence (⊨) of Γ (equivalently, if Γ "models" A).

#### The procedure:
1) **Preprocessing**: Transform Γ ⊨ A into an *equisatisfiable* Propositional Formula in Conjunctive Normal Form (CNF);
2) **Verify**: Evaluate *satisfiability* of the CNF using a CNF-SAT Solver (**CDCL**)


In [2]:
# Load the Problem from a File
filepath = 'examples/logcons/001.txt'

problem = LogConsProblem.from_file(filepath)
print(problem)

{
 !(A & B) | (C -> D) -> E
 A -> D
 !B | (E -> F)
} ⊨	(A & C) -> D



In [3]:
formula = problem.to_formula()

formula

'(!(A & B) | (C -> D) -> E) ∧ (A -> D) ∧ (!B | (E -> F)) ∧ ¬((A & C) -> D)'

In [4]:
f = ast.PropFormula.from_string(formula)
print(f"Input:\t{f}")
f2 = IFNF.to_IFNF(f)
print(f"IFNF:\t{f2}")
f3 = NNF.to_NNF(f2)
print(f"NNF:\t{f3}")
f4 = CNF.to_equisat_CNF(f3)
print(f"Equisat CNF:\t{f4}")
cnf = dimacs.DimacsCNF.from_ast(f4)
print(f"Dimacs CNF:\t{cnf}")

Input:	((((((¬(1 ∧ 2)) ∨ (3 → 4)) → 5) ∧ (1 → 4)) ∧ ((¬2) ∨ (5 → 6))) ∧ (¬((1 ∧ 3) → 4)))
IFNF:	(((((¬((¬(1 ∧ 2)) ∨ ((¬3) ∨ 4))) ∨ 5) ∧ ((¬1) ∨ 4)) ∧ ((¬2) ∨ ((¬5) ∨ 6))) ∧ (¬((¬(1 ∧ 3)) ∨ 4)))
NNF:	((((((1 ∧ 2) ∧ (3 ∧ (¬4))) ∨ 5) ∧ ((¬1) ∨ 4)) ∧ ((¬2) ∨ ((¬5) ∨ 6))) ∧ ((1 ∧ 3) ∧ (¬4)))
Equisat CNF:	(((((9 ∨ 5) ∧ ((¬1) ∨ 4)) ∧ ((¬2) ∨ ((¬5) ∨ 6))) ∧ ((1 ∧ 3) ∧ (¬4))) ∧ ((((¬9) ∨ 7) ∧ ((¬9) ∨ 8)) ∧ ((((¬8) ∨ 3) ∧ ((¬8) ∨ (¬4))) ∧ (((¬7) ∨ 1) ∧ ((¬7) ∨ 2)))))
Dimacs CNF:	[9, 5], [-1, 4], [-2, -5, 6], [1], [3], [-4], [-9, 7], [-9, 8], [-8, 3], [-8, -4], [-7, 1], [-7, 2]


In [5]:

try:
    cdcl = CDCL(
        cnf=cnf,
        options=CDCLOptions(
            heuristic=HeuristicType.VSIDS,
            restarts=True,
            forgets=True,
            timeout_seconds=120
        )
    )

    found = cdcl.solve()
except Exception as e:
    print(e)
    found = False

if found:
    assert cdcl.cnf.check(cdcl.v) # Verify that the found assignment satisfies the formula
    print(f"CNF is SAT:\n{cdcl.v}")
else:
    print("CNF is UNSAT")

There are not enough watchable literals to satisfy the clause clause [-1, 4]
CNF is UNSAT
